In [1]:
import pandas as pd
import re
import numpy as np
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from tensorflow.keras.preprocessing.text import Tokenizer
#from tensorflow.keras.utils import np_utils
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
from sklearn import tree
from tensorflow.keras.preprocessing import sequence
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn import metrics
from sklearn.svm import SVC
from sklearn import svm
from sklearn.feature_extraction.text import CountVectorizer

import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [23]:
from sklearn.tree import DecisionTreeClassifier

In [2]:
from google.colab import files
data_to_load = files.upload()
import io
dataset = pd.read_csv(io.BytesIO(data_to_load['amharic.csv']))

Saving amharic.csv to amharic.csv


In [3]:
dataset.head()

,labels,tweet,Unnamed: 2
0,Free,አስቀድሜ ጥያቄዬ በጨዋነት በውስጥ መስመር እንዲደርስዎ አድርጌ ፍትህን ለ...,NaN
1,Free\n,እነዚህን ወሳኝ ጉዳዮችን የሚያስፈፅም አካል እንዲቋቋምና ክትትል እንዲደ...,NaN
2,Free \n,የአማራ ህዝብ በአእምሮ ክንፉ ያልበረረበት ጥበብና ፍልስፍና ያልከፈተው የ...,NaN
3,Free\n,ከአማራ ህዝብ የሀገሪቱ ዘርፈ ብዙ እውቀት መንጭቶ የሞላበትከሙላቱም በመል...,NaN
4,Hate\n,ዛሬ በየትኛውም መለኪያ ይሁን መመዘኛ ኢትዮጵያዊነት የሚንፀባረቀው በአማራ...,NaN


In [4]:
tweet=dataset.tweet
labels=dataset.labels

In [5]:


dataset['labels']=[re.sub(r"\n", "", str(label)) for label in labels]

     


In [6]:





# drop function which is used in removing or deleting rows or columns from the CSV files
dataset.drop('Unnamed: 2', inplace=True, axis=1)

# display
print("\nCSV Data after deleting the column 'Unnamed: 2':\n")
print(dataset)



CSV Data after deleting the column 'Unnamed: 2':

      labels                                              tweet
0      Free   አስቀድሜ ጥያቄዬ በጨዋነት በውስጥ መስመር እንዲደርስዎ አድርጌ ፍትህን ለ...
1       Free   እነዚህን ወሳኝ ጉዳዮችን የሚያስፈፅም አካል እንዲቋቋምና ክትትል እንዲደ...
2      Free   የአማራ ህዝብ በአእምሮ ክንፉ ያልበረረበት ጥበብና ፍልስፍና ያልከፈተው የ...
3       Free  ከአማራ ህዝብ የሀገሪቱ ዘርፈ ብዙ እውቀት መንጭቶ የሞላበትከሙላቱም በመል...
4       Hate  ዛሬ በየትኛውም መለኪያ ይሁን መመዘኛ ኢትዮጵያዊነት የሚንፀባረቀው በአማራ...
...      ...                                                ...
29995   Hate         በአሉ የሁሉም ኢትዮጵያዊ ስላልሆነ በኦሮምኛው ቢለፋደድ ምን አገባን
29996   Free             ተባረክ አብቹ ፈር ቀዳጅ ስለሆንህ መጋረጃው መቀደድ ስለጀመረ
29997   Free   እስከ አሁን አንተ ብቻ ነው በ   መፅሀፍ ያልቻልከው  አንተም ታሪክ እ...
29998   Hate   ህገወጥት ጠቅላይ ሚንስትር ፅቤት የተፈቀደ ሆኖ ህዝብን እንዴት ህግ አክ...
29999   Hate  ደነዙ ጠቅላይ ሚንስትር ፅቤት ህገመንግስት ሳይሻሻል በህግ የተወሰነዉን የ...

[30000 rows x 2 columns]


In [8]:
words = dataset['tweet']
finalWords = []
for word in words:
    
    
    #remove any non-alphanumeric character
    word = re.sub(r'\W', ' ', str(word))
    
    #remove any digit
    word = re.sub("(\\d)+"," ",word)
    
    #remove punctuation 
    word = re.sub(",#'!:;"," ",word)
    
    #remove any latin characters 
    word = re.sub(r'[a-zA-Z]+', '', word)

    word = re.sub('\s+', ' ', word)  # remove extra whitespace
    finalWords.append(word)


In [9]:

tokenizedList = []
for lineWord in finalWords:
    separatedWords = lineWord.split(' ')
    word = [word for word in separatedWords if   word.strip() != '']
    tokenizedList.append(word)

In [10]:
# list amharic stop words
stopwords=['ነው', 'እና', 'እኔ', 'የእኔ','እኔ','ራሴ','እኛ','የእኛ','እኛ','ራሳችን','አንቺ' ,'የእርስዎ' ,'እራስዎ','እራሳችሁን','የእሱ',
           'እሱ','እ','ራሱ','እሷ' 'ራሷን','እሱ','በራሱ','እነሱ','የእነሱ','እራሳቸው','ምንድን','የት','ማን','ይሄ','ያንን','እነዚህ','እነዚያ'
           ,'ነው','እነዚህ','ናቸው','ነበር','ነበሩ','መሆን','ሆኗል','መሆን','አለ','ነበሩ','አለው','መስራት','ያደርገዋል','አደረጉት',
           'ማድረግ','እና','ግን','ከሆነ','ወይም','ምክንያቱም','እንደ','እስከ','ገና','የ','ለ','ከ','ጋር','ስለ','መካከል','ውስጥ',
           'ከዚህ','በፊት','በኋላ','ከላይ','ከታች','ወደ','ከ','ወደ','ላይ','ታች','ውስጥ','ውጭ','በላይ','በታች','በድጋሚ','እናስ','እናም']

           
cleanedWordsList= []
for lineWords in tokenizedList:
    tempList = []
    for word in lineWords:
        if word.strip() not in stopwords:
            tempList.append(word)
    cleanedWordsList.append(tempList)

In [11]:
# for the time being this is fine but convert it to something good but not this at all

normalizedSenteces = []
for linesWords in cleanedWordsList:
    tempList = []
    for word in linesWords:
        
        word = re.sub("ሐ","ሀ",word)
        word = re.sub("ሑ","ሁ",word)
        word = re.sub("ሒ","ሂ",word)
        word = re.sub("ሓ","ሃ",word)
        word = re.sub("ሔ","ሄ",word)
        word = re.sub("ሕ","ህ",word)
        word = re.sub("ሖ","ሆ",word)
        
        word = re.sub("ኀ","ሀ",word)
        word = re.sub("ኁ","ሁ",word)
        word = re.sub("ኂ","ሂ",word)
        word = re.sub("ኃ","ሃ",word)
        word = re.sub("ኄ","ሄ",word)
        word = re.sub("ኅ","ህ",word)
        word = re.sub("ኆ","ሆ",word)
        
        word = re.sub("ሠ","ሰ",word)
        word = re.sub("ሡ","ሱ",word)
        word = re.sub("ሢ","ሲ",word)
        word = re.sub("ሣ","ሳ",word)
        word = re.sub("ሤ","ሴ",word)
        word = re.sub("ሥ","ስ",word)
        word = re.sub("ሦ","ሶ",word)
        
        word = re.sub("ዐ","አ",word)
        word = re.sub("ዑ","ኡ",word)
        word = re.sub("ዒ","ኢ",word)
        word = re.sub("ዓ","ኣ",word)
        word = re.sub("ዔ","ኤ",word)
        word = re.sub("ዕ","እ",word)
        word = re.sub("ዖ","ኦ",word)
        
        word = re.sub("ዐ","አ",word)
        word = re.sub("ዑ","ኡ",word)
        word = re.sub("ዒ","ኢ",word)
        word = re.sub("ዓ","ኣ",word)
        word = re.sub("ዔ","ኤ",word)
        word = re.sub("ዕ","እ",word)
        word = re.sub("ዖ","ኦ",word)
        
        word = re.sub("ጸ","ፀ",word)
        word = re.sub("ጹ","ፁ",word)
        word = re.sub("ጺ","ፂ",word)
        word = re.sub("ጻ","ፃ",word)
        word = re.sub("ጼ","ፄ",word)
        word = re.sub("ጽ","ፅ",word)
        word = re.sub("ጾ","ፆ",word)
        
        tempList.append(word)
    normalizedSenteces.append(tempList)

In [12]:
# # Split into training and testing data
x = dataset['tweet']
y = dataset['labels']

In [14]:


x = np.array(dataset["tweet"])
y = np.array(dataset["labels"])
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
X = cv.fit_transform(x.astype('U')) 
#x = cv.fit_transform(x.apply(lambda x: np.str_(x)))
#X = cv.fit_transform(x)# Fit the Data
#Splitting the Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Split the training set into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)


     


In [15]:


print("Training set size:", X_train)
     


Training set size:   (0, 39510)	1
  (0, 105214)	1
  (1, 10680)	1
  (1, 1298)	1
  (1, 80957)	1
  (1, 54732)	1
  (1, 15876)	1
  (1, 100601)	1
  (1, 750)	1
  (1, 96505)	1
  (1, 8652)	1
  (1, 80024)	1
  (1, 80415)	1
  (1, 33332)	1
  (1, 96792)	1
  (1, 102380)	1
  (1, 98024)	1
  (1, 28242)	1
  (1, 105560)	1
  (1, 14727)	1
  (1, 54294)	1
  (1, 94490)	1
  (1, 43241)	1
  (1, 48168)	1
  (1, 78942)	1
  :	:
  (17998, 6760)	1
  (17998, 35720)	1
  (17998, 56540)	1
  (17998, 62401)	1
  (17998, 110149)	1
  (17998, 24521)	1
  (17998, 475)	1
  (17998, 12427)	1
  (17998, 108074)	1
  (17998, 13733)	1
  (17998, 49485)	1
  (17998, 35970)	1
  (17998, 14177)	1
  (17998, 23584)	1
  (17998, 11818)	1
  (17998, 30839)	1
  (17998, 9819)	1
  (17998, 18088)	1
  (17998, 14611)	1
  (17998, 66349)	1
  (17999, 8652)	1
  (17999, 80777)	1
  (17999, 63336)	1
  (17999, 54455)	1
  (17999, 92993)	1


In [16]:
print("Testing set size:",X_test)

Testing set size:   (0, 44755)	2
  (0, 6760)	1
  (0, 105895)	1
  (0, 15901)	1
  (0, 100569)	1
  (0, 111682)	1
  (0, 111623)	1
  (0, 99878)	1
  (0, 45440)	1
  (0, 17976)	1
  (0, 86487)	1
  (0, 35163)	1
  (0, 16818)	1
  (0, 56137)	1
  (0, 13795)	1
  (0, 31674)	1
  (0, 92475)	1
  (0, 55212)	1
  (0, 66744)	1
  (0, 108137)	1
  (0, 39399)	1
  (1, 44755)	1
  (1, 71642)	1
  (1, 49047)	1
  (1, 92082)	1
  :	:
  (5999, 49630)	1
  (5999, 109409)	1
  (5999, 111326)	1
  (5999, 27168)	1
  (5999, 104628)	1
  (5999, 13085)	1
  (5999, 81326)	1
  (5999, 111994)	1
  (5999, 25558)	1
  (5999, 39639)	1
  (5999, 41764)	2
  (5999, 15844)	1
  (5999, 59239)	1
  (5999, 8463)	1
  (5999, 6840)	1
  (5999, 13687)	1
  (5999, 8056)	1
  (5999, 7128)	1
  (5999, 69313)	1
  (5999, 112109)	1
  (5999, 70854)	1
  (5999, 82733)	1
  (5999, 10708)	1
  (5999, 66765)	1
  (5999, 23921)	1


In [17]:


print("Training set size:", X_val)
     


Training set size:   (0, 44755)	1
  (0, 63043)	1
  (0, 110317)	1
  (0, 63259)	1
  (0, 38022)	1
  (0, 29451)	1
  (0, 17841)	1
  (0, 62820)	1
  (0, 35714)	1
  (0, 52096)	1
  (0, 37666)	1
  (0, 46519)	1
  (0, 34527)	1
  (0, 109699)	1
  (0, 94976)	1
  (0, 76353)	1
  (0, 68940)	1
  (0, 6256)	1
  (0, 15444)	1
  (0, 68970)	1
  (0, 87604)	1
  (1, 78432)	1
  (1, 48255)	1
  (1, 36008)	1
  (1, 61343)	1
  :	:
  (5998, 40650)	1
  (5998, 57019)	1
  (5998, 64129)	1
  (5998, 57577)	1
  (5998, 77397)	1
  (5998, 3360)	3
  (5998, 25042)	1
  (5998, 77700)	1
  (5998, 80002)	1
  (5998, 11763)	1
  (5999, 82917)	1
  (5999, 44755)	3
  (5999, 107587)	1
  (5999, 9451)	1
  (5999, 81631)	1
  (5999, 22185)	1
  (5999, 58317)	3
  (5999, 39096)	1
  (5999, 83686)	1
  (5999, 8273)	1
  (5999, 82397)	1
  (5999, 32708)	1
  (5999, 52519)	2
  (5999, 2829)	1
  (5999, 90500)	1


In [18]:


X_train
     


<18000x112739 sparse matrix of type '<class 'numpy.int64'>'
	with 306573 stored elements in Compressed Sparse Row format>

In [19]:


y_train
     


array(['Hate', 'Free', 'Free', ..., 'Hate', 'Hate', 'Hate'], dtype=object)

In [20]:


X_test
     


<6000x112739 sparse matrix of type '<class 'numpy.int64'>'
	with 100858 stored elements in Compressed Sparse Row format>

In [21]:


y_test
     


array(['Free', 'Free', 'Free', ..., 'Hate', 'Hate', 'Hate'], dtype=object)

In [24]:


from sklearn.metrics import classification_report
#model.score(X_test,y_test)
# Define hyperparameters
max_depths = [67,2,5,9,4,6,100,1000] # maximum depth of the tree ,0
min_samples_splits = [1001,89,90,34,45,65,254] # minimum number of samples required to split an internal node 2
min_samples_leaves = [890,2001,2400,67,544,560,679,2] # minimum number of samples required to be at a leaf node 
# Train the model with different hyperparameters
for max_depth in max_depths:
    for min_samples_split in min_samples_splits:
        for min_samples_leaf in min_samples_leaves:
            # Define the model
            model = DecisionTreeClassifier(max_depth=max_depth, min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf,criterion='gini')
            # Train the model
            model.fit(X_train, y_train)
           #train and test
            score = model.score(X_test, y_test)
           
            # Print the results
            print(f"Max depth: {max_depth}, Min samples split: {min_samples_split}, Min samples leaf: {min_samples_leaf}, Accuracy score: {score}")

     


Max depth: 67, Min samples split: 1001, Min samples leaf: 890, Accuracy score: 0.5288333333333334
Max depth: 67, Min samples split: 1001, Min samples leaf: 2001, Accuracy score: 0.515
Max depth: 67, Min samples split: 1001, Min samples leaf: 2400, Accuracy score: 0.515
Max depth: 67, Min samples split: 1001, Min samples leaf: 67, Accuracy score: 0.5733333333333334
Max depth: 67, Min samples split: 1001, Min samples leaf: 544, Accuracy score: 0.5333333333333333
Max depth: 67, Min samples split: 1001, Min samples leaf: 560, Accuracy score: 0.5333333333333333
Max depth: 67, Min samples split: 1001, Min samples leaf: 679, Accuracy score: 0.5288333333333334
Max depth: 67, Min samples split: 1001, Min samples leaf: 2, Accuracy score: 0.5813333333333334
Max depth: 67, Min samples split: 89, Min samples leaf: 890, Accuracy score: 0.5288333333333334
Max depth: 67, Min samples split: 89, Min samples leaf: 2001, Accuracy score: 0.515
Max depth: 67, Min samples split: 89, Min samples leaf: 2400, A

In [27]:
# Test the model on the testing set
y_pred = model.predict(X_test)
            # Evaluate the model's performance
report = classification_report(y_test, y_pred)
print(report)

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

        Free       0.60      0.60      0.60      2986
       Free        0.00      0.00      0.00         1
        Hate       0.61      0.61      0.61      3013

    accuracy                           0.60      6000
   macro avg       0.40      0.40      0.40      6000
weighted avg       0.60      0.60      0.60      6000



/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [28]:
metrics.f1_score(y_test, y_pred, average='weighted', labels=np.unique(y_pred))

0.6050469925837445

# Predicting the outcome ***

In [ ]:
#Predicting the outcome
inp =input("Enter post :                 ")
dp = cv.transform([inp]).toarray()
dt=model.predict(dp)
if dt=='Hate Speech':
    print("//////////////////////////////")
    print("/////the post is blocked//////")
    print("//////////////////////////////")
elif dt=='Offensive Speech':
    print("Warnnig.....")
    print("Warnnig.....")
    print("Warnnig.....")
    print(inp)

else:
    print(inp)
   
print(dt)